# Importing our data from TMDB using API calls

## "Discover" Data Set

First we are getting our BIG set of movie data using the Discover API, with the following parameters:
    * include_adult false (does not include adult movies in the response)
    * include_video false (does not include short format video in the response)
    * page * where we are going to make calls to the maximum allowed number of pages which is 500
    * primary_release_date_gte 2010-01-01 (we want movie data from 2010 onward only)
    * primary_release_date_lte 2019-12-31 (this is a pre-covid case study)
    * vote_count.gte 31 (we want the movies to have at least 31 votes to be included)
    * with_original_language en (our client will make movies in English, and we will still have 10,000 returns)
    
The TMDB database allows for a maximum number of 10,000 returns, so we are trying to use parameter rules that will give us only the most relevant and important returns in order to make recommendations.

https://developers.themoviedb.org/3/discover/movie-discover

First loading in the libraries we will need to make these calls.

In [1]:
import json
import requests

Our API key is stored in our .secret folder in a json file. We're accessing that here and saving it to our api-key variable

In [2]:
def get_keys(path):
    '''docstring here'''
    with open(path) as f:
        return json.load(f)

keys = get_keys("/Users/Wadkins/Dropbox/Flatiron/module_01/.secret/tmdb_api.json")

api_key = keys['api_key']

In [3]:
response={}

for i in range (1,501):
    '''docstring here'''
    url = 'https://api.themoviedb.org/3/discover/movie?api_key={}&language=en-US&sort_by=popularity.desc&include_adult=false&include_video=false&page={}&primary_release_date.gte=2010-01-01&primary_release_date.lte=2019-12-31&vote_count.gte=31&with_original_language=en'.format(api_key, i)
    response[i] = (requests.get(url).json())

In [4]:
with open('tmdb_movies.json', 'w', encoding='utf-8') as f:
    json.dump(response, f, ensure_ascii=False, indent=4)

## IMDB ID Matchup

!!! STOP !!!
Don't run this portion of the notebook until AFTER the Discover data set created above ^ has been cleaned and exported in the main student.ipynb notebook

Then return to this notebook and run this section

In [5]:
# load our exported csv which is the dataframe we prepared in our main student.ipynb file
import pandas as pd
data = pd.read_csv('api_data/tmdb_discover.csv')

data

,popularity,vote_count,id,genre_ids,title,vote_average,release_date
0,823.042,12617,354912,"[16, 10751, 35, 12, 14, 10402]",Coco,8.2,2017-10-27
1,356.048,5124,474350,"[27, 14]",It Chapter Two,6.9,2019-09-04
2,333.745,15365,475557,"[80, 53, 18]",Joker,8.2,2019-10-02
3,279.613,6340,330457,"[16, 10751, 12, 35, 14]",Frozen II,7.3,2019-11-20
4,204.021,3720,316727,"[28, 27, 53]",The Purge: Election Year,6.4,2016-06-29
...,...,...,...,...,...,...,...
6462,0.600,40,184712,"[28, 27, 878]",Battledogs,5.0,2013-04-05
6463,0.600,322,174323,"[35, 18]",G.B.F.,5.9,2013-10-20
6464,0.600,41,173455,"[99, 10402]",Mistaken for Strangers,7.2,2013-09-19
6465,0.600,37,136568,"[28, 878]",Dragon Wasps,5.0,2012-09-17


In [40]:
#data = data.head(10)

In [6]:
# use the TMDB API to call for the IMDB ids and add them to the dataframe
response={}

for ind in data.index:
    i = data['id'][ind]
    url = 'https://api.themoviedb.org/3/movie/{}/external_ids?api_key={}'.format(i, api_key)
    response = (requests.get(url).json())
    data['id'][ind] = response['imdb_id']

C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
data

,popularity,vote_count,id,genre_ids,title,vote_average,release_date
0,823.042,12617,tt2380307,"[16, 10751, 35, 12, 14, 10402]",Coco,8.2,2017-10-27
1,356.048,5124,tt7349950,"[27, 14]",It Chapter Two,6.9,2019-09-04
2,333.745,15365,tt7286456,"[80, 53, 18]",Joker,8.2,2019-10-02
3,279.613,6340,tt4520988,"[16, 10751, 12, 35, 14]",Frozen II,7.3,2019-11-20
4,204.021,3720,tt4094724,"[28, 27, 53]",The Purge: Election Year,6.4,2016-06-29
...,...,...,...,...,...,...,...
6462,0.600,40,tt2457138,"[28, 27, 878]",Battledogs,5.0,2013-04-05
6463,0.600,322,tt2429074,"[35, 18]",G.B.F.,5.9,2013-10-20
6464,0.600,41,tt2737310,"[99, 10402]",Mistaken for Strangers,7.2,2013-09-19
6465,0.600,37,tt2170371,"[28, 878]",Dragon Wasps,5.0,2012-09-17


We have succesfully converted all of our TMDB ids into IMDB ids!

Time to re-export this dataframe so we can load it back into the student notebook.

In [8]:
#exporting the csv
data.to_csv('api_data/tmdb_discover_converted.csv', index=False)